# Vega-Altair - a surprisingly powerful Python library for plotting interactive maps

This notebook runs the user through the interactive altair chart example in the medium blog article. See the readme for this folder for the link to the medium blog. 

## 1. Set up

### 1.1 Loading packages/functions

In [11]:
import pandas as pd
import altair as alt
from getters.english_la_iod_data_2019 import get_english_la_iod_2019
from getters.english_lsoa_iod_data_2019 import get_english_lsoa_iod_2019
from getters.lsoa_shapefiles_2011 import get_english_lsoa_shapefiles_2011
from utils.utils_fonts_colours import *
from PIL import Image
import os

### 1.2 Loading in the data

In [12]:
imd_lsoa_data = get_english_lsoa_iod_2019().query("lad19nm == 'York'")
geodata_lsoa = get_english_lsoa_shapefiles_2011("yorkshire_and_the_humber")

## 2. Creating the map

For this example we want to produce a map of York, so we need to filter the IoD data to look at York Local Authority.

In [13]:
iod_lsoa_data_york = imd_lsoa_data.query("lad19nm == 'York'")
lsoas_to_plot = list(iod_lsoa_data_york.lsoa11nm)


We can now set up the colour scheme for the map:

In [14]:
color_lsoa = alt.Color(
    "income_deprivation_affecting_children:O", 
    scale = alt.Scale(scheme="yellowgreenblue"), 
    title="IDACI", 
    legend = alt.Legend(orient="top")
    )

To plot a geojson in Vega-Altair we use .mark_geoshape()

In [15]:
choro_lsoa = (
    alt.Chart(geodata_lsoa)
    .mark_geoshape(
        stroke="black"
    )
    # We then need to link the geojson to the iod_lsoa_data_york dataframe, using the .transform_lookup we can do this in the plotting code.
    .transform_lookup(
        # We want to match on the lsoa11nm which is stored under properties.lsoa11nm in the geojson
        lookup="properties.lsoa11nm",
        from_=alt.LookupData(
            # This is the dataframe we want to read the data from.
            iod_lsoa_data_york,
            # This is the column we want to match on
            "lsoa11nm",
            # These are the column names we want to bring in from the iod_lsoa_data_york dataframe.
            ["lsoa11cd", "lsoa11nm", "lad19cd", "lad19nm", "income_deprivation_affecting_children"],
        ),
    )
    # We then want to make sure that we just plot the LSOAs which are in the York dataset, rather than the whole of the UK, therefore we use this .transform_filter function.
    .transform_filter(
        alt.FieldOneOfPredicate(
            field="properties.lsoa11nm", oneOf=lsoas_to_plot
        )
    )
    # Here is where we encode the colour, which we specified earlier.
    .encode(
        color=color_lsoa,
        # We’ve also added an interactive tooltip so you can hover over each LSOA and see its name and its IDACI.
        tooltip=[alt.Tooltip("lsoa11nm:N", title="LSOA"), alt.Tooltip("income_deprivation_affecting_children:O", title="IDACI")])
        .project(type="identity", reflectY=True)
        .properties(width=500,height=500)
)

Finally, we have some configurations for the map e.g. changing font size. 

In [16]:
choro_lsoa = (
                choro_lsoa.configure_legend(
                labelLimit = 0,
                titleLimit = 0,
                titleFontSize = 13,
                labelFontSize = 13,
                symbolStrokeWidth = 1.5,
                symbolSize=150
                ).configure_view(
                    strokeWidth = 0
                ).configure_axis(
                    labelLimit=0,
                    titleLimit=0)
            )

And with that, the first step is done - you have a choropleth map!

In [17]:
choro_lsoa

alt.Chart(...)

## 3. Creating the bar charts

Next up, we’re need to create the bar chart. We’re going to plot a bar chart showing the average IDACI for the whole of York. This is just setting up the chart so we can add in our selections further down. This allows for the bar chart when combined with map, to always have the average IDACI of York present to compare to the LSOA's selected. But is not necessary for this sort of interactivity! You could also combine the bar chart with the selections only (bar_chart).

First, we create a pandas DataFrame with two columns which are the same as the columns we’re using in the choropleth map “lsoa11nm” and “income_deprivation_affecting_children”. The values of these columns are simply “York average” and the calculated mean to decimal places, respectively.

In [18]:
iod_lsoa_data_york_av = pd.DataFrame(
    {"lsoa11nm":["York average"], 
    "income_deprivation_affecting_children":[f"{iod_lsoa_data_york.income_deprivation_affecting_children.mean():.2f}"]
    })


We can now make the bar chart containing the average:

In [19]:

bar_chart_york_average = (alt.Chart(iod_lsoa_data_york_av)
   .mark_bar(color="#0000FF")
   .encode(
       x=alt.X("income_deprivation_affecting_children:Q",
           title="IDACI",
           axis=alt.Axis(tickMinStep=1),
       ),
       y=alt.Y(
           "lsoa11nm:N",
           title="LSOA Name",
       ),
   )
   .properties(width=400, height=400)
)


In [20]:
bar_chart_york_average

alt.Chart(...)

So now we have two figures - an incredibly simple bar chart and a choropleth - and we are now going to enable them to interact with each other.

We do this using the “selections” part of the Vega-Altair library. There are many options, we could use point selection, single selection or multi selection. We are going to use multi selection which will enable users to hold down “Shift” and select as many LSOAs as they like which will then appear on the bar chart.

However, we want the bar chart to be originally empty, rather than show every LSOA in York, so we need a separate selection for this. If you don't want the bar chart to initialise empty, you can just use lsoa_select only. 

In [21]:
lsoa_select = alt.selection_multi(fields=["lsoa11nm"])
lsoa_select_empty = alt.selection_multi(fields=["lsoa11nm"], empty = "none")

We also want the colour of the LSOA which are selected to remain as their IDACI colour and we want the remaining LSOAs to turn to grey. We therefore have to rewrite our previous code for the colour scheme to reflect this.

In [22]:
color_lsoa = alt.condition(
    lsoa_select, 
    alt.Color(
    "income_deprivation_affecting_children:O", 
    scale = alt.Scale(scheme="yellowgreenblue"), 
    title="IDACI", legend = alt.Legend(orient="top")
    ), 
    alt.value("lightgray")
    )

Using alt.condition, we are adding a conditional selection onto the colour. We’re telling Vega-Altair that on a selection choice, colour the LSOAs selected using the original colour scale, and if they’re not in that selection, make them light grey.

We can now add the selector to the initial choropleth map:

In [23]:
choro_lsoa = (
               alt.Chart(geodata_lsoa)
               .mark_geoshape(
                   stroke="black"
               )
               .transform_lookup(
                   lookup="properties.lsoa11nm",
                   from_=alt.LookupData(
                       data=iod_lsoa_data_york,
                       key="lsoa11nm",
                       fields=["lsoa11cd", "lsoa11nm", "lad19cd", "lad19nm", "income_deprivation_affecting_children"],
                   ),
               )
               .transform_filter(
                   alt.FieldOneOfPredicate(
                       field="properties.lsoa11nm", oneOf=lsoas_to_plot
                   )
               )
               .encode(
                   color=color_lsoa,
                   tooltip=[alt.Tooltip("lsoa11nm:N", title="LSOA"), alt.Tooltip("income_deprivation_affecting_children:O", title="IDACI")]).add_selection(lsoa_select, lsoa_select_empty).project(type="identity", reflectY=True).properties(width=500,height=500))


We can now click on LSOA's in the chart below:

In [24]:
choro_lsoa

alt.Chart(...)

We now create a bar chart that will allow filtering based on what LSOA is selected on the map.

In [25]:
bar_chart = (alt.Chart(iod_lsoa_data_york)
               .mark_bar(color="#0000FF")
               .encode(
                   x=alt.X("income_deprivation_affecting_children:Q",
                       title="IDACI",
                       axis=alt.Axis(tickMinStep=1),
                   ),
                   y=alt.Y(
                       "lsoa11nm:N",
                       title="LSOA Name",
                   ),
               )
               .transform_filter(lsoa_select_empty)
               .properties(width=400, height=400)
           )


We also want to combine the two bar charts we have created so that we have the York average IDACI on the bar chart. This is very simple to do with a '+':

In [26]:
bar_charts_combined = bar_chart_york_average + bar_chart

Given we have added the selector (which depends on what you click on the map), this figure will not plot till combined with the map. 

## 4. Combining the charts 

Finally, to combine the charts, we use alt.vconcat(). 

In [27]:
combined_charts = (
    alt.vconcat(choro_lsoa, bar_charts_combined, center=True)
    .configure_legend(
    
        labelLimit=0,
        titleLimit=0,
        titleFontSize=13,
        labelFontSize=13,
        symbolStrokeWidth=1.5,
        symbolSize=150,
        )
    .configure_view(
        strokeWidth=0
    )
    .configure_axis(
        labelLimit=0,
        titleLimit=0,
    )
)


And ta da 🎉, you now have an interactive choropleth map!

You can hold shift and click on LSOA's in York. Selecting the LSOA's on the map will show them on the bar chart below to compare the IDACI deciles. You can double-click to remove the all the selections and hover over the map to see the deciles.

In [28]:
combined_charts

alt.VConcatChart(...)